#### Colab run

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/MyDrive/master_thesis/pi-gan_sequential

/content/drive/.shortcut-targets-by-id/16udig9ZMaNcASs5Maj6kv7tg-TL3PnSE/Master Thesis/pi-gan_sequential


In [3]:
!pip install kornia
!pip install torchinfo

     |████████████████████████████████| 286kB 4.3MB/s 


#### End Colab run

In [4]:
import warnings
import time

from py_files.args import *

from py_files.functions import *

from py_files.dataset import *

from py_files.cnn_model import *
from py_files.pigan_model import *

from py_files.load_utils import *
from py_files.data_utils import *
from py_files.plot_utils import *
from py_files.train_utils import *
from py_files.save_utils import *

Imported CNN and Mapping functions.


/usr/local/lib/python3.7/dist-packages/kornia/augmentation/augmentation.py:1833: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,


#### Train the model

In [5]:
def train():  
    
    warnings.filterwarnings("ignore")
    
    ##### path to wich the model should be saved #####
    path = get_folder(ARGS)
    DEVICE = set_device(ARGS)
    
    with open(os.path.join(path, "ARGS.txt"), "w") as f:
        print(vars(ARGS), file=f)
        
    ##### data preparation #####
    train_dl, val_dl, test_dl = initialize_dataloaders(ARGS, DEVICE)
    print("train batch:", next(iter(train_dl))[1][:5])
    print("eval batch:", next(iter(val_dl))[1][:5])
    print("test batch:", next(iter(test_dl))[1][:5])
            
    ##### initialize models and optimizers #####
    models, optims, schedulers = initialize_models_and_optims(ARGS, DEVICE)
    
    blur_layer = initialize_blurring_layer(1.0, DEVICE)
    
    ##### load pretrained model #####
    load_from_saved_run(models, optims, DEVICE, ARGS)

    ##### loss function #####
    criterions = [nn.BCELoss(), nn.MSELoss()]
        
    ##### epoch, train loss mean, train loss std, val loss mean, val loss std #####
    mask_losses, pcmra_losses, dice_losses = np.empty((0, 5)), np.empty((0, 5)), np.empty((0, 5))
    
    for ep in range(ARGS.pcmra_epochs):
    
        t = time.time() 

        for model in models.values():
            model.train()

        loss, _ = train_model(train_dl, models, optims, schedulers, criterions[1], blur_layer, ARGS, output="pcmra")
        
        if ep % ARGS.eval_every == 0: 

            print(f"Epoch {ep} took {round(time.time() - t, 2)} seconds.")
            
            t_pcmra_mean, t_pcmra_std, _, _ = \
                val_model(train_dl, models, criterions[1], blur_layer, ARGS, output="pcmra")
            v_pcmra_mean, v_pcmra_std, _, _ = \
                val_model(val_dl, models, criterions[1], ARGS, output="pcmra")

            pcmra_losses = np.append(pcmra_losses, [[ep ,t_pcmra_mean, t_pcmra_std, 
                                         v_pcmra_mean, v_pcmra_std]], axis=0)
            
            save_loss(path, pcmra_losses, models, optims, name="pcmra_loss", 
                      save_models=True)
        
    
    for ep in range(ARGS.mask_epochs):
    
        t = time.time() 

        for model in models.values():
            model.train()

        loss, _ = train_model(train_dl, models, optims, schedulers, criterions[0], blur_layer, ARGS, output="mask")
        
        if ep % ARGS.eval_every == 0: 

            print(f"Epoch {ep} took {round(time.time() - t, 2)} seconds.")
            
            t_mask_mean, t_mask_std, t_dice_mean, t_dice_std = \
                val_model(train_dl, models, criterions[0], blur_layer, ARGS, output="mask")
            v_mask_mean, v_mask_std, v_dice_mean, v_dice_std = \
                val_model(val_dl, models, criterions[0], blur_layer, ARGS, output="mask")

            mask_losses = np.append(mask_losses, [[ep ,t_mask_mean, t_mask_std, 
                                         v_mask_mean, v_mask_std]], axis=0)
            dice_losses = np.append(dice_losses, [[ep ,t_dice_mean, t_dice_std, 
                                         v_dice_mean, v_dice_std]], axis=0)
            
            save_loss(path, mask_losses, models, optims, name="mask_loss", 
                      save_models=True)
            save_loss(path, dice_losses, models, optims, name="dice_loss", 
                      save_models=False)

## Run as .ipynb

In [ ]:
for cnn_setup, mapping_setup in [(-1, -1)]:

    ARGS = init_ARGS()
        
    ARGS.pcmra_epochs = 0
    
    ARGS.patience = 100 
    
    ARGS.flip = True 
    ARGS.crop = True 
    ARGS.rotate = True 
    ARGS.translate = True

    ARGS.sdf = False 
    ARGS.sdf_split = None
    
    print(vars(ARGS))

    train()  

    torch.cuda.empty_cache()    

{'device': 'GPU', 'print_models': False, 'name': '', 'pretrained': None, 'pretrained_best_dataset': 'train', 'pretrained_best_loss': 'mask', 'pretrained_models': None, 'pretrained_lr_reset': None, 'dataset': 'new', 'seed': 34, 'rotate': True, 'translate': True, 'translate_max_pixels': 20, 'flip': True, 'crop': True, 'stretch': True, 'stretch_factor': 1.2, 'norm_min_max': [0, 1], 'pcmra_epochs': 0, 'mask_epochs': 5000, 'batch_size': 24, 'eval_every': 50, 'shuffle': True, 'n_coords_sample': 5000, 'min_lr': 1e-05, 'cnn_setup': -1, 'pcmra_train_cnn': True, 'mask_train_cnn': True, 'mapping_setup': -1, 'dim_hidden': 256, 'siren_hidden_layers': 3, 'first_omega_0': 30.0, 'hidden_omega_0': 30.0, 'pcmra_first_omega_0': 30.0, 'pcmra_hidden_omega_0': 30.0, 'cnn_lr': 0.0001, 'cnn_wd': 0, 'mapping_lr': 0.0001, 'pcmra_mapping_lr': 0.0001, 'siren_lr': 0.0001, 'siren_wd': 0, 'pcmra_siren_lr': 0.0001, 'pcmra_siren_wd': 0, 'patience': 100, 'sdf': False, 'sdf_split': None}
saved_runs/pi_gan_2021_06_25_10_

## Run as .py

In [ ]:
def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

if __name__ == "__main__":

    PARSER = argparse.ArgumentParser()

    
    # Arguments for training
    PARSER.add_argument('--device', type=str, default="GPU", 
                        help='Device that should be used.')

    PARSER.add_argument('--print_models', type=str2bool, nargs="?", const=True, default=False, 
                        help='Print the models after initialization or not.')

    PARSER.add_argument('--name', type=str, default="", 
                        help='Name of the folder where the output should be saved.')
    
    

    # pretrained params 
    
    PARSER.add_argument('--pretrained', type=str, default=None, 
                        help='Folder name of pretrained model that should be loaded.')
    
    PARSER.add_argument('--pretrained_best_dataset', type=str, default="train", 
                        help='Pretrained model with lowest [train, val] loss.')
    
    PARSER.add_argument('--pretrained_best_loss', type=str, default="mask", 
                        help='Pretrained model with lowest [train, val] loss.')
    
    PARSER.add_argument('--pretrained_models', type=str, default=None, 
                        help='Choose which pretrained models to load. None = all models')
    
    PARSER.add_argument('--pretrained_lr_reset', type=str, default=None, 
                        help='Reset the lr to a value.')
    
    
    
    # data
    PARSER.add_argument('--dataset', type=str, default="new", 
                        help='The dataset which we train on.')
    
    PARSER.add_argument('--seed', type=int, default=34, 
                        help='Seed for initializig dataloader')
    
    PARSER.add_argument('--rotate', type=str2bool, nargs="?", const=True, default=True, 
                        help='Rotations of the same image')
    
    PARSER.add_argument('--translate', type=str2bool, nargs="?", const=True, default=True, 
                        help='Translations of the same image')
    
    PARSER.add_argument('--translate_max_pixels', type=int, default=20, 
                        help='Translation max in height and width.')
    
    PARSER.add_argument('--flip', type=str2bool, nargs="?", const=True, default=True, 
                        help='Flips the train image')
    
    PARSER.add_argument('--crop', type=str2bool, nargs="?", const=True, default=True, 
                        help='Crops the train image')

    PARSER.add_argument('--stretch', type=str2bool, nargs="?", const=True, default=True, 
                        help='Stretches the train image')

    PARSER.add_argument('--stretch_factor', type=float, default=1.2, 
                        help='Stretch maximum of the train image')

    PARSER.add_argument('--norm_min_max', type=list, default=[0, 1], 
                        help='List with min and max for normalizing input.')
    
    
    
    # train variables
    PARSER.add_argument('--pcmra_epochs', type=int, default=5000, 
                        help='Number of epochs for pcmra training.')

    PARSER.add_argument('--mask_epochs', type=int, default=5000, 
                        help='Number of epochs for mask training.')
    
    PARSER.add_argument('--batch_size', type=int, default=24, 
                        help='Number of epochs.')
        
    PARSER.add_argument('--eval_every', type=int, default=50, 
                        help='Set the # epochs after which evaluation should be done.')
    
    PARSER.add_argument('--shuffle', type=str2bool, nargs="?", const=True, default=True, 
                        help='Shuffle the train dataloader?')
    
    PARSER.add_argument('--n_coords_sample', type=int, default=5000, 
                        help='Number of coordinates that should be sampled for each subject.')
    
    PARSER.add_argument('--min_lr', type=float, default=1e-5, 
                        help='Minimum lr, input for lr scheduler.')
    
    
    
    # CNN
    PARSER.add_argument('--cnn_setup', type=int, default=-1, 
                        help='Setup of the CNN.')
    
    PARSER.add_argument('--pcmra_train_cnn', type=str2bool, nargs="?", const=True, default=True, 
                        help='Whether to also train the cnn during pcmra reconstruction.')

    PARSER.add_argument('--mask_train_cnn', type=str2bool, nargs="?", const=True, default=True, 
                        help='Whether to also train the cnn during mask segmentation.')


    
    # Mapping
    PARSER.add_argument('--mapping_setup', type=int, default=-1, 
                        help='Setup of the Mapping network.')

    
    
    # SIREN
    PARSER.add_argument('--dim_hidden', type=int, default=256, 
                        help='Dimension of hidden SIREN layers.')
    
    PARSER.add_argument('--siren_hidden_layers', type=int, default=3, 
                        help='Number of hidden SIREN layers.')
    
    
    PARSER.add_argument('--first_omega_0', type=float, default=30., 
                        help='Omega_0 of first layer.')
    
    PARSER.add_argument('--hidden_omega_0', type=float, default=30., 
                        help='Omega_0 of hidden layer.')
    
    
    PARSER.add_argument('--pcmra_first_omega_0', type=float, default=30., 
                        help='Omega_0 of first layer of PCMRA siren.')
    
    PARSER.add_argument('--pcmra_hidden_omega_0', type=float, default=30., 
                        help='Omega_0 of hidden layer of PCMRA siren.')
    
    
    
    # optimizers
    PARSER.add_argument('--cnn_lr', type=float, default=1e-4, 
                        help='Learning rate of cnn optim.')

    PARSER.add_argument('--cnn_wd', type=float, default=0, 
                        help='Weight decay of cnn optim.')

    
    PARSER.add_argument('--mapping_lr', type=float, default=1e-4, 
                        help='Learning rate of siren optim.')
    
    PARSER.add_argument('--pcmra_mapping_lr', type=float, default=1e-4, 
                        help='Learning rate of siren optim.')
    

    PARSER.add_argument('--siren_lr', type=float, default=1e-4, 
                        help='Learning rate of siren optim.')

    PARSER.add_argument('--siren_wd', type=float, default=0, 
                        help='Weight decay of siren optim.')
    
    
    PARSER.add_argument('--pcmra_siren_lr', type=float, default=1e-4, 
                        help='Learning rate of PCMRA siren optim.')    
    
    PARSER.add_argument('--pcmra_siren_wd', type=float, default=0, 
                        help='Weight decay of PCMRA siren optim.')
    
    PARSER.add_argument('--patience', type=int, default=200, 
                        help='Patience of the LR scheduler.')
    
    
    ARGS = PARSER.parse_args()
    
    train()